In [45]:
import os
import pandas as pd
from dotenv import load_dotenv
from FiinQuantX import FiinSession


load_dotenv()
username = os.getenv('username')
password = os.getenv('password')


client = FiinSession(
    username=username,
    password=password,
).login()

data = data = client.Fetch_Trading_Data(
    realtime = False, 
    tickers = 'VIX',
    fields = ['open', 'high', 'low', 'close', 'volume', 'bu','sd', 'fb', 'fs', 'fn'], 
    by = '1d',
    period = 300,
).get_data()

print(data)

Fetching data, it may take a while. Please wait...
    ticker         timestamp          open          high           low  \
0      VIX  2024-06-20 00:00  13827.119394  13827.119394  13408.115776   
1      VIX  2024-06-21 00:00  13446.207014  13560.480728  13331.933300   
2      VIX  2024-06-24 00:00  13331.933300  13408.115776  12646.291016   
3      VIX  2024-06-25 00:00  12798.655968  12912.929682  12684.382254   
4      VIX  2024-06-26 00:00  12798.655968  12989.112158  12722.473492   
..     ...               ...           ...           ...           ...   
295    VIX  2025-08-25 00:00  36450.000000  36450.000000  32300.000000   
296    VIX  2025-08-26 00:00  32450.000000  34850.000000  32300.000000   
297    VIX  2025-08-27 00:00  36050.000000  37250.000000  35500.000000   
298    VIX  2025-08-28 00:00  37000.000000  38450.000000  35850.000000   
299    VIX  2025-08-29 00:00  38800.000000  39250.000000  37650.000000   

            close      volume          bu          sd       

In [46]:
fi = client.FiinIndicator()
data['sma_20'] = fi.sma(data['close'], window = 20)
data['sma_50'] = fi.sma(data['close'], window = 50)
data['sma_200'] = fi.sma(data['close'], window = 200)
print(data[['close', 'sma_20', 'sma_50', 'sma_200']])

            close   sma_20      sma_50      sma_200
0    13408.115776      NaN         NaN          NaN
1    13331.933300      NaN         NaN          NaN
2    12722.473492      NaN         NaN          NaN
3    12798.655968      NaN         NaN          NaN
4    12798.655968      NaN         NaN          NaN
..            ...      ...         ...          ...
295  32600.000000  30582.5  21790.4812  13529.02002
296  34850.000000  31127.5  22237.0000  13652.79282
297  36850.000000  31690.0  22722.0000  13785.61322
298  38400.000000  32245.0  23241.0000  13926.42172
299  37700.000000  32815.0  23744.0000  14064.20642

[300 rows x 4 columns]


In [ ]:
class backtest_strategy:
    def __init__(self, data):
        self.data = data
        self.position = 0  # 1 for holding a position, 0 for no position
        self.cash = 100000  # starting cash
        self.shares = 0  # number of shares held
        self.trades = []  # record of trades

    def run(self):
        for i in range(1, len(self.data)):
            if self.data['sma_20'].iloc[i] > self.data['sma_50'].iloc[i] and self.position == 0:
                # Buy signal
                self.shares = self.cash // self.data['close'].iloc[i]
                self.cash -= self.shares * self.data['close'].iloc[i]
                self.position = 1
                self.trades.append((self.data.index[i], 'BUY', self.shares, self.data['close'].iloc[i]))
                print(f"[BUY] Date: {self.data['timestamp'].iloc[i]}, Shares: {self.shares}, Price: {self.data['close'].iloc[i]}, Cash left: {self.cash}")
            elif self.data['sma_20'].iloc[i] < self.data['sma_50'].iloc[i] and self.position == 1:
                # Sell signal
                self.cash += self.shares * self.data['close'].iloc[i]
                self.position = 0
                self.trades.append((self.data.index[i], 'SELL', self.shares, self.data['close'].iloc[i]))
                print(f"[SELL] Date: {self.data['timestamp'].iloc[i]}, Shares: {self.shares}, Price: {self.data['close'].iloc[i]}, Cash after sale: {self.cash}")
                self.shares = 0

        # Final portfolio value
        final_value = self.cash + (self.shares * self.data['close'].iloc[-1])
        return final_value, self.trades

bt = backtest_strategy(data)
final_value, trades = bt.run()
print(f"Final Portfolio Value: {final_value}")

[BUY] Date: 2024-09-30 00:00, Shares: 8.0, Price: 11428.8, Cash left: 8569.600000000006
[SELL] Date: 89, Shares: 8.0, Price: 10571.64, Cash after sale: 93142.72
[BUY] Date: 2025-02-19 00:00, Shares: 8.0, Price: 10666.88, Cash left: 7807.680000000008
Final Portfolio Value: 309407.68
